In [ ]:
!pip install -q unipka py3Dmol

In [ ]:
import unipka
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np

calc = unipka.UnipKa(use_simple_smarts=False)

Get microstate distribution at target pH

In [ ]:
mol = Chem.MolFromSmiles("[H]OC(=O)C([H])([H])N([H])[H]")
distribution_df = calc.get_distribution(mol, pH=2)
distribution_df

You can pass in either a SMILES string or RDKit molecule. If the molecule has 3D coordinates, all enumerated microstates will be transplanted onto the reference, and the placement of any new hydrogens will be optimised. In the viewer below, we can see only the hydrogens coordinates differ between all microstates.

This is particularly useful when i) determing the correct protomer from a crystal structure, as only heavy atoms are visible, and ii) determing the correct protomer from a Boltz structure, as only heavy atoms are predicted.

In [ ]:
import py3Dmol

mol = Chem.MolFromSmiles("N1=CC=CC=C1NCCC")
AllChem.EmbedMolecule(mol)
distribution_df = calc.get_distribution(mol, pH=7.4)

viewer = py3Dmol.view(width=800, height=400, viewergrid=(1,2))

mols = distribution_df.mol.tolist()
for m in mols:
    mol1_block = Chem.MolToMolBlock(m)
    viewer.addModel(mol1_block, 'mol', viewer=(0,0))
    viewer.setStyle({'stick': {}}, viewer=(0,0))
                   # viewer=(0,0))

viewer.zoomTo(viewer=(0,0))
viewer


Calculate basic pKa from either manually or automatically enumerated microstates

In [ ]:
pka_manual = calc.get_macro_pka_from_macrostates(acid_macrostate=["c1cc[nH+]cc1"], base_macrostate=["c1ccncc1"])
pka_auto = calc.get_basic_micro_pka("c1ccncc1", idx=3)
assert np.isclose(pka_manual,pka_auto)

acidic_pka_of_conjugate_acid = calc.get_acidic_micro_pka("c1cc[nH+]cc1", idx=3)

assert np.isclose(acidic_pka_of_conjugate_acid,pka_manual)
pka_manual

Get dominant microstate at a specific pH

In [ ]:
calc.get_dominant_microstate("N1=CC=CC=C1", pH=7.4)

Visualise the microstate distribution using the Jupyter widget

In [ ]:
calc.draw_distribution("N1=CC=CC=C1NC", mode="jupyter")

Visualise the microstate distribution using the matplotlib

In [ ]:
calc.draw_distribution("N1=CC=CC=C1NC", mode="matplotlib")

Calculate logD

In [ ]:
calc.get_logd("N1=CC=CC=C1", pH=5.)

Draw logD distribution

In [ ]:
calc.draw_logd_distribution("N1=CC=CC=C1", mode="matplotlib")

Get the state penalty (the free energy required to shift the ionization equilibrium toward the neutral non-zwitterionic state). This can be used to calculate properties like following the methodology of [Rowan Sci and Lawrenz](https://chemrxiv.org/engage/api-gateway/chemrxiv/assets/orp/resource/item/68388349c1cb1ecda02ba65d/original/physics-informed-machine-learning-enables-rapid-macroscopic-p-ka-prediction.pdf).

In [ ]:
state_penalty, reference_microstates_df  = calc.get_state_penalty("c1ccncc1", pH=5.)
print(state_penalty, "kcal/mol")
# Selects formally neutral microstates that minimize atom-centered charges, preferring non-zwitterionic forms over zwitterionic counterparts.
reference_microstates_df

Predict the probability that Kpuu > 0.3. This won't run in colab, as it needs the conda package `xtb`.

In [ ]:
# Known drug with low CNS penetrance has a very low probability
fexofenadine = "CC(C)(C1=CC=C(C=C1)C(CCCN2CCC(CC2)C(C3=CC=CC=C3)(C4=CC=CC=C4)O)O)C(=O)O"
prob = calc.predict_brain_penetrance(fexofenadine)
prob